<a href="https://colab.research.google.com/github/Arun-nexus/deep_learning/blob/main/name_entity_recognizattion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Arun\Downloads\NER dataset.csv\NER dataset.csv",encoding="latin1")
df.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
df['Sentence #'] = df['Sentence #'].ffill()
agg_func = lambda s: list(s.values)
grouped = df.groupby("Sentence #").agg({"Word": agg_func, "Tag": agg_func})
print(grouped.head())


                                                              Word  \
Sentence #                                                           
Sentence: 1      [Thousands, of, demonstrators, have, marched, ...   
Sentence: 10     [Iranian, officials, say, they, expect, to, ge...   
Sentence: 100    [Helicopter, gunships, Saturday, pounded, mili...   
Sentence: 1000   [They, left, after, a, tense, hour-long, stand...   
Sentence: 10000  [U.N., relief, coordinator, Jan, Egeland, said...   

                                                               Tag  
Sentence #                                                          
Sentence: 1      [O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...  
Sentence: 10     [B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...  
Sentence: 100    [O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...  
Sentence: 1000                   [O, O, O, O, O, O, O, O, O, O, O]  
Sentence: 10000  [B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...  


In [ ]:
all_tokens=grouped["Word"]
tags=grouped["Tag"]

In [ ]:
dict={"<pad>":0,"<unk>":1}
for row in all_tokens:
    for word in row :
        if word not in dict:
            dict[word]=len(dict)

In [ ]:
tags_dict={"<pad>":0,"<unk>":1}
for row in tags:
    for tag in row:
        if tag not in tags_dict:
            tags_dict[tag]=len(tags_dict)

In [ ]:
tags_dict

{'<pad>': 0,
 '<unk>': 1,
 'O': 2,
 'B-geo': 3,
 'B-gpe': 4,
 'B-tim': 5,
 'B-org': 6,
 'I-geo': 7,
 'B-per': 8,
 'I-per': 9,
 'I-org': 10,
 'I-tim': 11,
 'B-art': 12,
 'I-art': 13,
 'B-nat': 14,
 'I-gpe': 15,
 'I-nat': 16,
 'B-eve': 17,
 'I-eve': 18}

In [ ]:
sequence=[]
for row in all_tokens:
    token=[dict.get(word,dict["<unk>"]) for word in row]
    sequence.append(token)


In [ ]:
vocab_size=len(sequence)

In [ ]:
tags_sequence=[]
for row in tags:
    tags_token=[tags_dict.get(word,tags_dict["<unk>"]) for word in row]
    tags_sequence.append(tags_token)

In [ ]:
import torch
tensor_sequence=[torch.tensor(row) for row in sequence]
tensor_tags=[torch.tensor(row) for row in tags_sequence]

In [ ]:
from torch.utils.data import DataLoader,TensorDataset
from torch.nn.utils.rnn import pad_sequence
def padding(word):
    word=pad_sequence(word,batch_first=True)
    return word
paded_data=padding(tensor_sequence)
paded_tags=padding(tensor_tags)

In [ ]:
paded_tags=paded_tags[:10000,:]
paded_data=paded_data[:10000,:]
print(paded_data.shape)
print(paded_tags.shape)
max_len=104

torch.Size([10000, 104])
torch.Size([10000, 104])


In [ ]:
import torch.nn as nn
import math
class positional_encoding(nn.Module):
    def __init__(self,d_model,max_len):
        super().__init__()
        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len).unsqueeze(1)
        div_term=torch.exp(torch.arange(0,d_model,2))*(-math.log(10000.0)/d_model)
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        pe=pe.unsqueeze(0)
        self.register_buffer("pe",pe)

    def forward(self,x):
        return x+self.pe[:,:x.size(1)]

In [ ]:
class multihead_attention(nn.Module):
    def __init__(self,d_model,num_heads):
        super().__init__()
        assert d_model%num_heads==0

        self.d_k=d_model//num_heads

        self.q_liner=nn.Linear(d_model,d_model)
        self.k_linear=nn.Linear(d_model,d_model)
        self.v_linear=nn.Linear(d_model,d_model)
        self.out=nn.Linear(d_model,d_model)
        self.num_heads=num_heads

    def forward(self,q,k,v,mask=None):
        batch_size=q.size(0)
        def transform(x,linear):
            x=linear(x)
            x=x.view(batch_size,-1,self.num_heads,self.d_k)
            return x.transpose(1,2)
        q,k,v=transform(q,self.q_liner),transform(k,self.k_linear),transform(v,self.v_linear)
        scores=torch.matmul(q,k.transpose(-2,-1))/math.sqrt(self.d_k)
        if mask is not None:
            scores=scores.masked_fill(mask==0,1e9)
        attn=torch.softmax(scores,dim=-1)
        output=torch.matmul(attn,v)
        output=output.transpose(1,2).contiguous().view(batch_size,-1,self.d_k,self.num_heads)
        return self.out(output)

In [ ]:
class feedforward(nn.Module):
    def __init__(self,d_model,d_ff=2048,drop=0.2):
        super().__init__()
        self.feature=nn.Sequential(
            nn.Linear(d_model,d_ff),
            nn.Dropout(drop),
            nn.Linear(d_ff,d_model)
        )
    def forward(self,x):
        return self.feature(x)


In [ ]:
class trasnformerencoder(nn.Module):
    def __init__(self,d_model,num_heads,d_ff,drop=0.2):
        super().__init__()
        self.attn=multihead_attention(d_model,num_heads)
        self.ff=feedforward(d_model,d_ff,drop)
        self.norm1=nn.LayerNorm(d_model)
        self.norm2=nn.LayerNorm(d_model)
        self.dropout=nn.Dropout(drop)

    def forward(self,x,mask=None):
        x2=self.norm1(x+self.dropout(self.attn(x,x,x,mask)))
        x3=self.ff(x2)
        x=self.norm2(x2+self.dropout(self.attn(x3,x3,x3,mask)))
        return x

In [ ]:
class encoder(nn.Module):
    def __init__(self,vocab_size,d_model,max_len,num_layers,num_heads,d_ff,num_classes):
        super().__init__()
        self.embedding=nn.Embedding(vocab_size,d_model)
        self.pe=positional_encoding(d_model,max_len)
        self.layer=nn.ModuleList(trasnformerencoder(d_model,num_heads,d_ff,0.2) for _ in range(num_layers))
        self.norm=nn.LayerNorm(d_model)
        self.fc=nn.Linear(d_model,num_classes)

    def forward(self,src,mask=None):
        x=self.embedding(src)
        x=self.pe(x)
        for layer in self.layer:
            x=layer(x,mask)
            x= self.norm(x)

            return self.fc(x)

In [ ]:
device=("cuda" if torch.cuda.is_available() else "cpu")
model=encoder(vocab_size=vocab_size,d_model=512,max_len=max_len,num_layers=6,num_heads=8,d_ff=2048,num_classes=len(tags_dict)).to(device)
model_loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)
scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode="min",factor=0.5,patience=2)

In [ ]:
import torch.nn as nn
class early_stopping(nn.Module):
    def __init__(self,min_delta,patience):
        super().__init__()
        self.min_delta=min_delta
        self.partience=patience
        self.early_stop=False
        self.best_loss=float("inf")
        self.counter=0

    def __call__(self,val_loss):
        if self.min_delta >= val_loss-self.best_loss:
            self.counter=0
            self.best_loss=val_loss
        else:
            self.counter+=1
            if self.counter>=self.partience:
                self.early_stop=True

In [ ]:
from torch.amp import GradScaler,autocast

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
dataset=TensorDataset(paded_data,paded_tags)
from sklearn.model_selection import train_test_split
train_data,val_data=train_test_split(dataset,test_size=0.3,random_state=42)
train_data_loader=DataLoader(train_data,batch_size=128)
val_data_loader=DataLoader(val_data,batch_size=128)

In [ ]:
epochs=50
stopper=early_stopping(1e-4,4)
training_loss_store=[]
validation_loss_store=[]
training_accuracy_store=[]
validation_accuracy_store=[]
scaler=GradScaler()

for epoch in range(epochs):
    running,total,correct=0,0,0
    for x,y in train_data_loader:
        x,y=x.to(device),y.to(device)

        optimizer.zero_grad()
        with autocast("cuda"):
            output=model(x)
            loss=model_loss(output,y)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        running+=loss.item()
        _,pred=torch.max(output,1)
        correct+=(pred==y).sum().item()
        total+=y.size(0)
        # print(torch.isnan(pred).any(), torch.isnan(y).any())

    training_loss=running/len(y)
    training_acc=correct/total
    training_loss_store.append(training_loss)
    training_accuracy_store.append(training_acc)

    model.eval()
    vrunning,vtotal,vcorrect=0,0,0
    for x,y in val_data_loader:
        x,y=x.to(device),y.to(device)

        optimizer.zero_grad()
        with autocast("cuda"):
            voutput=model(x)
            vloss=model_loss(voutput,y)

        vrunning+=vloss.item()
        _,vpred=torch.max(voutput,1)
        vcorrect+=(vpred==y).sum().item()
        vtotal+=y.size(0)
    val_loss=vrunning/len(y)
    val_acc=vcorrect/vtotal
    validation_loss_store.append(val_loss)
    validation_accuracy_store.append(val_acc)

    print(f"epoch: {epoch+1} training_loss: {training_loss:.4f} training acc:{training_acc:.2f} validation_loss: {val_loss:.4f} validation_acc: {val_acc:.2f} ")
    stopper(val_loss)
    if stopper.early_stop:
        print("early_stopping triggered")
        break